# 보만사 전체 댓글수집

In [84]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time
import pandas as pd # pandas 라이브러리 임포트

# 옵션 설정
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
# Chrome 버전을 명시하는 것보다 WebDriverManager가 자동으로 최신 버전을 찾게 하는 것이 좋습니다.
# options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")

# WebDriverManager를 사용하여 ChromeDriver 자동 설치 및 설정
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# 네이버 카페 로그인 페이지로 이동
driver.get("https://cafe.naver.com/f-e/cafes/19997929/menus/0?q=%EC%8B%9C%ED%97%98&t=1750149314419")
wait = WebDriverWait(driver, 20)

# 로그인 버튼 클릭
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gnb_login_button > span.gnb_txt'))).click()

# ID 입력
id_input_element = wait.until(EC.presence_of_element_located((By.ID, 'id')))
driver.execute_script("arguments[0].focus();", id_input_element)
time.sleep(1) # 포커스 후 이벤트 처리를 위한 대기
driver.execute_script("arguments[0].value = '아이디';", id_input_element)

# PW 입력
pw_input_element = wait.until(EC.presence_of_element_located((By.ID, 'pw')))
driver.execute_script("arguments[0].focus();", pw_input_element)
time.sleep(1) # 포커스 후 이벤트 처리를 위한 대기
driver.execute_script("arguments[0].value = '비밀번호';", pw_input_element)
time.sleep(2) # 입력 완료 후 대기

# 로그인 버튼 클릭
driver.find_element(By.CSS_SELECTOR, "#log\.login").click()
time.sleep(3) # 로그인 후 페이지 전환 대기

# 모든 게시글 데이터를 저장할 리스트
all_posts_data = []

# 이제부터 게시글 수집 및 페이지네이션 처리
while True:
    print("\n--- 새로운 페이지 로딩 ---")
    # 1. 현재 페이지의 게시글들을 순회하며 데이터 수집
    # 네이버 카페의 게시글 목록은 보통 한 페이지에 15개 또는 20개입니다.
    # 정확한 개수를 확인하고 range 값을 조정하세요. (예: 1부터 21까지 for i in range(1, 21))
    for i in range(1, 16): # 1부터 15까지 (총 15개 게시글 가정)
        try:
            # 게시글 버튼 찾기 (안정성을 위해 클릭 가능할 때까지 기다림)
            article_selector = f"tbody > tr:nth-child({i}) > td:nth-child(2) > div > div > a.article"
            article_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, article_selector)))
            article_button.click()
            time.sleep(2) # 게시글 로딩 대기

            # 게시글 내용이 있는 iframe으로 전환 (cafe_main)
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "cafe_main")))

            # content와 user 요소 찾기 및 텍스트 추출
            content_selector = "#app > div > div > div.ArticleContentBox > div.article_container > div:nth-child(1)"
            user_selector = "#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div.profile_area > div.profile_info > div"
            
            content_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, content_selector)))
            content = content_element.text
            
            user_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, user_selector)))
            user = user_element.text
            
            print(f"사용자: {user}\n내용:\n{content}")
            


            # 추출된 데이터를 리스트에 추가
            all_posts_data.append({"User": user, "Content": content}) 
            print("--- 게시글 수집 완료 ---\n")

            # 이전 페이지 (게시글 목록)로 돌아가기
            driver.back()
            time.sleep(2) # 뒤로가기 후 페이지 로딩 대기

        except Exception as e:
            # 게시글을 찾지 못했거나 클릭할 수 없는 경우 (현재 페이지의 마지막 게시글에 도달)
            print(f"게시글 {i} 수집 중 오류 발생 또는 현재 페이지의 마지막 게시글에 도달: {e}")
            break # 현재 페이지의 게시글 순회를 중단하고 페이지네이션으로 넘어갑니다.
            
    # --- 페이지네이션 처리 ---
    try:
        # 현재 보이는 페이지 번호 버튼들 수집
        page_numbers_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".Pagination .btn.number")))
        
        next_page_button_to_click = None

        # 현재 페이지 번호 (활성화된 버튼) 찾기
        current_page_num = -1
        for btn in page_numbers_elements:
            if btn.get_attribute("aria-pressed") == "true":
                current_page_num = int(btn.text)
                break
        
        # 현재 페이지 세트 내에서 다음 페이지 번호가 있으면 클릭
        found_next_in_set = False
        for btn in page_numbers_elements:
            try:
                btn_num = int(btn.text)
                if btn_num == current_page_num + 1:
                    next_page_button_to_click = btn
                    found_next_in_set = True
                    break
            except ValueError:
                # '...' 등의 텍스트가 있는 버튼은 int 변환 실패하므로 건너뜀
                continue
        
        # 현재 페이지 세트의 마지막 페이지(예: 10페이지)이고 '다음 페이지 세트' 버튼(>)이 있다면 그것을 클릭
        if not found_next_in_set:
            try:
                # 다음 페이지 세트 버튼 (꺽쇠 모양)
                next_set_button = driver.find_element(By.CSS_SELECTOR, ".Pagination .btn.type_next")
                
                # '다음' 버튼이 비활성화되어 있는지 확인
                if "disabled" in next_set_button.get_attribute("class"):
                    print("마지막 페이지 세트입니다. 더 이상 페이지가 없습니다.")
                    break # 더 이상 이동할 페이지가 없으므로 루프 종료
                
                next_page_button_to_click = next_set_button
                print("다음 페이지 세트로 이동합니다.")
                
            except Exception:
                # 다음 세트 버튼이 아예 없는 경우 (마지막 페이지이거나 10페이지 이하)
                print("더 이상 페이지를 찾을 수 없습니다. (다음 세트 버튼 없음)")
                break

        if next_page_button_to_click:
            wait.until(EC.element_to_be_clickable(next_page_button_to_click)).click()
            print(f"페이지 이동: {next_page_button_to_click.text if next_page_button_to_click.text else '다음 세트'}")
            time.sleep(3) # 페이지 로딩 대기 시간을 충분히 줍니다.
        else:
            print("더 이상 이동할 페이지가 없어 종료합니다.")
            break

    except Exception as e:
        print(f"페이지네이션 처리 중 오류 발생 또는 마지막 페이지 도달: {e}")
        break # 페이지네이션 처리 중 문제가 발생하면 루프 종료

# WebDriver 종료
driver.quit()


### CSV 파일로 데이터 추출

# 현재 날짜와 시간을 포함하는 파일명 생성
now = datetime.now()
file_name = f"naver_cafe_posts_{now.strftime('%Y%m%d_%H%M%S')}.csv"

# 리스트를 Pandas DataFrame으로 변환
df = pd.DataFrame(all_posts_data)

# CSV 파일로 저장 (인코딩을 'utf-8-sig'로 설정하여 엑셀에서 한글 깨짐 방지)
df.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"\n총 {len(all_posts_data)}개의 게시글 데이터를 '{file_name}'으로 저장했습니다.")


--- 새로운 페이지 로딩 ---
사용자: 라라네옷장
내용:
1. 먼저 이 글의 제목을 변경 해 주세요 : 지역+회사명+보험설계사+닉네임
네
(예시) 수원 ㅇㅇ보험회사/ ㅇㅇGA 보험설계사 뽀빠이 가입인사 합니다.
2. 카페 활동 닉네임 (영어이신 분은 한글로 바꾸어주세요) :
네
3. 소속 보험회사 :
더베스트
4. 보험경력 :
4년
5. 보만세에서 얻고 싶은 것 :
정보
6. 자기소개(경력, 영업방법 등) :
안녕하세요
삼성생명원수사에서 시작해서
현재 대리점에서 영업하고있습니다
경력은 5년차이지만..
아직 부족한게많고 이겨내야될께많고 배워야할께많은 설계사입니다
7. 가입경로 [ 예) 검색(검색어 입력-보험설계사 등등), 소개, 기타 ] :
네이버검색 손해보험시험모의고사
●필독 - 회원등업
https://cafe.naver.com/gustn34264/16449
가입인사글1개+댓글20개 정회원 자동등업
★등업혜택 : 필수자료 150개★
https://cafe.naver.com/gustn34264/56557
● 카톡 채널을 "추가"하시면
유용한 정보와 안내를 받아 보실 수 있습니다.
(카톡에서 '보만세'검색 또는
채널링크 접속 : http://pf.kakao.com/_rtxjyb )
● 위 질문 미 입력시 다시 신입회원으로 강등.
★이 달의 보험강의 모음★
https://cafe.naver.com/gustn34264/53907
📢보만세 광고안내
https://cafe.naver.com/gustn34264/16547
※※※ 신규 회원에게 불법으로 리크루팅 쪽지/메일을 보내는 분들이
있습니다. 불법광고로 입사한 회원들이 100% 피해를 보고 있습니다.
내용 캡쳐해서 제게 문자 주시면 조치하고 커피쿠폰 드리겠습니다.
운영자 조이파파 010-5582-7573※※※
--- 게시글 수집 완료 ---

사용자: 곰돌이222
내용:
※제목앞에 말머리를 꼭 달아주세요 (질문 / 고민 / 일상/ 영업일기)
내일시험본다해서 왕부담중입니다
설계시험..말그대로..
저위에 순환계주요치

페이지 이동: 2

--- 새로운 페이지 로딩 ---
사용자: 인카GOAT송준호지점장
내용:
12편. 거절을 관리하는 대화법
상담을 하다 보면
가장 많이 듣게 되는 말이 있다.
“생각 좀 해볼게요.”
“괜찮아요.”
“다음에 연락드릴게요.”
그 순간, 마음이 철렁 내려앉는다.
‘또 거절당했구나.’
‘뭐가 부족했던 걸까?’
‘다시는 연락하지 말라는 뜻일까?’
그렇게 상담 후 연락을 끊고,
기록조차 남기지 못하고
무거운 마음만 남은 채
또 하나의 관계가 스쳐 지나간다.
거절은 관계의 끝이 아니다
대부분의 경우에 우리는
거절을 ‘실패’라고 받아들인다.
하지만 사실 거절은
관계의 한 단계일 뿐이다.
거절은 ‘아니오’가 아니라
‘아직은 아니오’일 수 있다.
관계를 맺어가는 과정에서
고객은 검토하고, 판단하고, 방어하는 시간을 갖는 것이다.
그 시간 안에
우리는 무너질 수도 있고, 연결될 수도 있다.
차이는 오직 하나,
거절을 어떻게 대하느냐다.
거절을 바로 정리하면, 다음 기회는 없다
거절을 들은 뒤 바로 명단에서 제외하거나,
상담 기록을 지워버리는 경우가 많다.
하지만 사실 그 순간이
관계를 더 깊게 만들 수 있는 기회일 수도 있다.
거절 직후의 한 마디,
거절 이후의 한 통의 메시지,
그게 상담보다 더 강한 신뢰를 남기기도 한다.
“괜찮습니다”는 무책임한 말이 될 수 있다
고객이 거절했을 때
무의식적으로 이렇게 말할 때가 있다.
“아 네, 괜찮습니다. 나중에 필요하시면 연락 주세요.”
이 말은 예의는 있지만
아무런 연결도 남기지 않는 말이다.
괜찮다고 했지만,
사실 나도, 고객도 서로 불편한 채로 끝나는 대화다.
이럴 땐 조금 다르게 말해보자.
“고객님께 꼭 필요한 시점이 아니었다면,
당연히 망설이실 수 있다고 생각해요.
혹시 나중에라도 보험 관련해서 궁금한 거 있으시면
제가 자료만이라도 편하게 전달드릴 수 있어요.”
이건 관계를 열어두는 말이다.
거절은 피드백이다. 감정으로 듣지 말고, 정보로 읽자
거절을 들었을 때
“난 거절당했다”는 감정에 빠지기 쉽다.
하지만 

사용자: 금융주치의 송닥터
내용:
※제목앞에 말머리를 꼭 달아주세요 (질문 / 고민 / 일상/ 영업일기)
생명 보험쪽과 더불어, 손보를 교차판매하고 싶은데요..
절차나 어디가 좋은지 정보가 전혀없어서요...
선배님들 도와주세요~~~~
6월에 시험보고 바로 위촉 진행하고 싶습니다.~~~
--- 게시글 수집 완료 ---

사용자: 평택lol 남매맘
내용:
※제목앞에 말머리를 꼭 달아주세요 (질문 / 고민 / 일상/ 영업일기)
타사근무하고.퇴사하면서.가족보험 싹 정리하고.나왔거든요
경력으로 입사할려고했더니.유지율때문에
미끄러졌어요
전속생보로 해야지할지
아님 손보시험보고 교차로.생보로갈지 고민이됩니다
--- 게시글 수집 완료 ---

사용자: 보만세스텝
내용:
[단독]조직화된 설계사 대리시험…결국 보험협회가 고소, 73명 적발
보험설계사 자격 시험 희망자를 모은 뒤 이들을 대신해 현직 설계사에게 대리 시험을 치게 한 혐의로 법인보험대리점(GA·General Agency) 대표 등 보험업계 종사(희망)자 73명이 경찰에 적발됐다.
4일 경찰과 손해보험협회 등에 따르면 경기남부경찰청 반부패·경제범죄수사대는 자격시험 주체인 보험협회에 대한 업무방해, 주민등록증 등 공문서부정행사, 보험업법 위반 등 혐의로 업계 10위권 GA 대표 A씨 등 73명을 지난 4월 수원지검 안산지청에 불구속 송치했다. GA는 한 보험사에 속하지 않고 여러 보험사의 상품을 비교·분석해 판매하는 대리점을 의미한다.
A씨 등은 지난 2022년 4월부터 지난해 7월까지 생명보험협회와 손해보험협회가 주관하는 보험설계사 자격시험 응시자를 모으고 시험 합격 경험이 있는 현직 설계사들에게 대리시험을 보게 한 혐의를 받는다. 시험 응시료는 1회당 2만원인데, A씨 등은 대리시험을 쳐주면서 10만~15만원의 수수료를 받은 것으로 조사됐다.
출처 : 중앙일보
기사전문링크
https://www.joongang.co.kr/article/25341481
--- 게시글 수집 완료 ---

사용자: 진노Be전
내용:
1. 먼

사용자: 셀리아초희
내용:
※제목앞에 말머리를 꼭 달아주세요 (질문 / 고민 / 일상/ 영업일기)
안녕하세요.
이번에 손보와 제3보험 합격했습니다
생보시험 앞두고 있는데요
합격후 코드는 어떻게 받는건가요?
합격후 코드받고 사정상 11월쯤 시작해야하는데
코드만받고 한동안 일을안해도 코드유지는
되는건가요?
답변부탁드립니다.
--- 게시글 수집 완료 ---

사용자: 알토스 전유나
내용:
안녕하세요? 알토스 전유나입니다.
QnA 시작하겠습니다. 실제 초보 설계사님의 고민입니다. 고수분들의 댓글 부탁드립니다.^^
Q ) 설계사 등록 된지 한달도 안 된 초보 설계사입니다.
보장분석 화법 상품 미숙지 상태에서 지인들에게 알리고만 있는 상태입니다.
미숙지이니 지인에게 자신감 없는 태도가 보입니다.
새벽부터 공부하기는 하지만 단번에 흡수하기엔 매우 시간이 부족합니다.
10콜3방은 커녕 3콜1방 정도하고 그나마 보험얘기도 못하고 겨우 알리는 정도 입니다.
환산 30맞추는 것조차 두려운데
이 시점에서 제가 무엇을 어떻게 해야할까요??바쁘시겠지만 알려주신다면 정말 감사하겠습니다
A)
질문에서 많은 고민이 느껴 지네요...
시험만 합격하면 고민이 끝난 줄 알았는데... 코드가 나오니 이제 부터 진정한 고민 시작이시죠?
대부분의 사람들은 코드 등록 후에 막막함을 느끼고, 심지어 고민만 하다가 포기하는 사람도 많습니다.
제가 신입때 했던 방법을 공유해 보겠습니다.
1. TA
- 10콜 3방은 목표치 일뿐입니다. 안된다고 자존감을 잃으실 필요가 없다는 말씀!! ㅎㅎㅎ
콜을 시작할때 마음은 " 아님 말고, 싫음 말고 " 이런 마음을 가지세요. 나는 왜 안되지? 라고 생각하는 순간
전화기 버튼 누르는게 어려워 집니다. 경험치가 쌓이면 자연스럽게 미팅 성공 확률도 올라가실 겁니다.
웃는 얼굴로 콜을 하세요. 전화기 넘어로 우리의 기분이 모두 전달됩니다.
어떤 디비인지 모르겠지만, 어떤 것이든 TA의 목적은 계약이 아니라 만남이 목적입니다.
기-승-전- 미팅 으로 가야 합니다. 미팅 약속을 잡

사용자: 드뎌
내용:
부산에서 손해보험 제3보험 시험장소문의입니다
6월17일 접수는했다고하는데 장소와 시간은 안나왓다고합니다
그건 언제 알수있나요??
만냑아직안나온거면 보통 어디서 몇시에 치나요??
--- 게시글 수집 완료 ---

사용자: 뚜이12
내용:
안녕하세요! 얼마전부터 설계매니저 준비하고 있는대요
보험쪽 관련 지식이 하나도 없어서 막막하네요 ㅠㅠ
손보랑 생명 자격증 따서 그때 회사에 들어가려고 하는대
시험등록을 하려고 하면 무조건 회사에서 등록을 해줘야한다고 하는거 같더라구여..
돈은 벌어야 하고.. 그러면 회사로 바로 들어가면 설계매니저한다고 하면 되나요?? 그럼 시험 보는 과정부터 도오ㅏ주시나요?? 선배 매니저분들 ㅠㅠ 부탁드립니다
--- 게시글 수집 완료 ---

사용자: 춘식이라이언
내용:
※제목앞에 말머리를 꼭 달아주세요 (질문 / 고민 / 일상/ 영업일기)
처음 시험58점으로 불합격하고
현타쎄게 오고..계속 55 51
월요일 드디어 합격했어요🥹🥹🥹🥹
영상은 안들어오고
기출문제 마니 푸는게 답인가봐요🙏🏻
--- 게시글 수집 완료 ---

사용자: 구리댁
내용:
※제목앞에 말머리를 꼭 달아주세요 (질문 / 고민 / 일상/ 영업일기)
안녕하세요....
궁금한 점이 있어 글을 올립니다.
올해 4월에 손+제3 자격이 사라졌다고 하는게 맞는지 모르겠지만...
코드를 내지 못해서 다시 시험을 봐야하는데요.
현재는 생명소속입니다.
재시험 절차에 대해서 아시는 분 계시다면 알려주세요...
--- 게시글 수집 완료 ---

사용자: 노강이
내용:
1. 먼저 이 글의 제목을 변경 해 주세요 : 지역+회사명+보험설계사+닉네임
(예시) 수원 ㅇㅇ보험회사/ ㅇㅇGA 보험설계사 뽀빠이 가입인사 합니다.
2. 카페 활동 닉네임 (영어이신 분은 한글로 바꾸어주세요) : 노깡
3. 소속 보험회사 : 한국보험금융
4. 보험경력 : 5년
5. 보만세에서 얻고 싶은 것 : 자료가 필요 합니다.
6. 자기소개(경력, 영업방법 등) : 브리핑 영업으로 시작해서 대면 영업까지 4년

...

사용자: 한국이러닝협회
내용:
안녕하세요. 한국이러닝협회입니다.
내일배움카드로 저렴하게 수강할 수 있는 보험업계에서 필수 자격증으로 손꼽히는
손해사정사 교육과정을 소개합니다. :)
손해사정사는 보험 사고에 대한 정확한 손해를 산정하고 보상 처리를 담당하는 전문가로서,
보험개발원에서 주관하고 있는 자격증 시험을 통해 자격을 부여받을 수 있습니다.
보험업계에서 종사하시는 분들뿐만 아니라, 보험사, 손해사정법인, 로펌 보험 관련 부서 등에서
경력을 쌓고 전문가로 활동하기를 희망하시는 분들이 선호하고 많이들 취득하시는 자격증입니다.
앞으로의 전망이 밝은 유망 직종이지만, 자격증을 취득하기 위해서 치러야 하는 시험이
학습량이 방대하고 복잡한 이론들을 공부해야해서 많은 분들이 어려움을 겪고 계시죠.
이러한 손해사정사 자격증 준비! 국민내일배움카드로도 할 수 있다는 사실, 알고 계셨나요?
한국이러닝협회는 실업자(구직자)와 근로자(재직자) 모두가 수강할 수 있는 강의를 운영하고 있습니다!
게다가 이러닝 교육이기 때문에, 시간과 장소의 구애를 받지 않고
수강생들의 스케줄에 따라서 유연하게 수강하실 수 있답니다:)
수강생들에게는 각 과정에 해당하는 교재도 무료로 제공해드리고 있어요!
손해사정사 1차 시험에서는 손해사정이론, 보험계약법, 보험업법을 공통과목으로 시행하고 있는데요,
한국이러닝협회에서는 이 3가지 과목 모두 운영중에 있으며 할인된 금액으로 수강 가능합니다!
빠른 합격으로 가는 지름길!
국비지원을 받아 비용에 대한 부담은 덜고
전문강사 선생님과 함께하는 손해사정사 시험 준비, 지금 바로 도전해보세요!
[한국이러닝협회 사이트 바로가기]
https://jobgo.ne.kr
성적우수자 이벤트 매 달 성적 우수자 10명을 선정하여 2만원 상당의 기프티콘 을 보내드립니다! 매달 훈련 종료된 학습자 중에서 상위 10명 선정 (동점자의 경우 평가 횟수가 많은 학습자 선정) 자세히보기 1 2 3 4 정지 국민내일배움카드 실업자/근로자 개강일 12월 05일(목요일) 수강신청 12월 0

사용자: 릴리맘
내용:
지금 조회해보니 아직 없네요 어제 1시쯤 시험 봤어요~불합격하면 조회도 안되나요? 미리 댓글 감사합니다~
--- 게시글 수집 완료 ---

사용자: 쩌빕
내용:
바로 안나오나요?
빨리 확인하고 싶은데 😅
--- 게시글 수집 완료 ---

사용자: 째니
내용:
1. 먼저 이 글의 제목을 변경 해 주세요 : 지역+회사명+보험설계사+닉네임
(예시) 수원 ㅇㅇ보험회사/ ㅇㅇGA 보험설계사 뽀빠이 가입인사 합니다.
2. 카페 활동 닉네임 (영어이신 분은 한글로 바꾸어주세요) : 째니
3. 소속 보험회사 : 없음
4. 보험경력 : 신입
5. 보만세에서 얻고 싶은 것 : 보험설계사 자격증을 준비하고있어서 정보를 얻고싶어서 가입했습니다!
6. 자기소개(경력, 영업방법 등) : 아직 자격증 준비중이라..ㅠㅠ
7. 가입경로 [ 예) 검색(검색어 입력-보험설계사 등등), 소개, 기타 ] : 손해보험 시험 준비로 검색하다가 알게되었습니다
●필독 - 회원등업
https://cafe.naver.com/gustn34264/16449
가입인사글1개+댓글20개 정회원 자동등업
★등업혜택 : 필수자료 150개★
https://cafe.naver.com/gustn34264/56557
● 카톡 채널을 "추가"하시면
유용한 정보와 안내를 받아 보실 수 있습니다.
(카톡에서 '보만세'검색 또는
채널링크 접속 : http://pf.kakao.com/_rtxjyb )
● 위 질문 미 입력시 다시 신입회원으로 강등.
★이 달의 보험강의 모음★
https://cafe.naver.com/gustn34264/53907
📢보만세 광고안내
https://cafe.naver.com/gustn34264/16547
※※※ 신규 회원에게 불법으로 리크루팅 쪽지/메일을 보내는 분들이
있습니다. 불법광고로 입사한 회원들이 100% 피해를 보고 있습니다.
내용 캡쳐해서 제게 문자 주시면 조치하고 커피쿠폰 드리겠습니다.
운영자 조이파파 010-5582-7573※※※
--- 게시글 수집 완료 ---

사용

사용자: 까꿍TV
내용:
1. 먼저 이 글의 제목을 변경 해 주세요 : 지역+회사명+보험설계사+닉네임
(예시) 수원 ㅇㅇ보험회사/ ㅇㅇGA 보험설계사 뽀빠이 가입인사 합니다.
2. 카페 활동 닉네임 (영어이신 분은 한글로 바꾸어주세요) :
3. 소속 보험회사 : 미정
4. 보험경력 : 없음
5. 보만세에서 얻고 싶은 것 : 유용한정보
6. 자기소개(경력, 영업방법 등) : 생명시험 준비중이며 개척영업할 생각입니다
7. 가입경로 [ 예) 검색(검색어 입력-보험설계사 등등), 소개, 기타 ] : 검색
●필독 - 회원등업
https://cafe.naver.com/gustn34264/16449
가입인사글1개+댓글20개 정회원 자동등업
★등업혜택 : 필수자료 150개★
https://cafe.naver.com/gustn34264/56557
● 카톡 채널을 "추가"하시면
유용한 정보와 안내를 받아 보실 수 있습니다.
(카톡에서 '보만세'검색 또는
채널링크 접속 : http://pf.kakao.com/_rtxjyb )
● 위 질문 미 입력시 다시 신입회원으로 강등.
★이 달의 보험강의 모음★
https://cafe.naver.com/gustn34264/53907
📢보만세 광고안내
https://cafe.naver.com/gustn34264/16547
※※※ 신규 회원에게 불법으로 리크루팅 쪽지/메일을 보내는 분들이
있습니다. 불법광고로 입사한 회원들이 100% 피해를 보고 있습니다.
내용 캡쳐해서 제게 문자 주시면 조치하고 커피쿠폰 드리겠습니다.
운영자 조이파파 010-5582-7573※※※
--- 게시글 수집 완료 ---

사용자: 만능워킹맘
내용:
일하다가 그만두고 다시 일하려니까 시험을 또쳐야 되네요..
칠패로 메리츠사옥에서 치는데
주변에 주차할만한곳 있을까요?
맛집은...? ㅋㅋㅋㅋ
서울까지 가는데... 뭐라도 먹고 올까 싶어서요
고생한 나에게 주는 .. 뭐 그런..ㅋㅋㅋㅋ
도와주세요 서울 사시는 분들~~~
--- 게시글 수집 완료 ---

사용자: 미미로그
내용:

사용자: 살리도
내용:
1. 먼저 이 글의 제목을 변경 해 주세요 : 지역+회사명+보험설계사+닉네임
(예시) 수원 ㅇㅇ보험회사/ ㅇㅇGA 보험설계사 뽀빠이 가입인사 합니다.
2. 카페 활동 닉네임 (영어이신 분은 한글로 바꾸어주세요) :
3. 소속 보험회사 : 청주 MG
4. 보험경력 : 아직 경력은 없습니당
5. 보만세에서 얻고 싶은 것 : 문제 / 정보
6. 자기소개(경력, 영업방법 등) : 시험 준비중입니다 ^^
7. 가입경로 [ 예) 검색(검색어 입력-보험설계사 등등), 소개, 기타 ] : 검색! 보험설계사 기출문제
●필독 - 회원등업
https://cafe.naver.com/gustn34264/16449
가입인사글1개+댓글20개 정회원 자동등업
★등업혜택 : 필수자료 150개★
https://cafe.naver.com/gustn34264/56557
● 카톡 채널을 "추가"하시면
유용한 정보와 안내를 받아 보실 수 있습니다.
(카톡에서 '보만세'검색 또는
채널링크 접속 : http://pf.kakao.com/_rtxjyb )
● 위 질문 미 입력시 다시 신입회원으로 강등.
★이 달의 보험강의 모음★
https://cafe.naver.com/gustn34264/53907
📢보만세 광고안내
https://cafe.naver.com/gustn34264/16547
※※※ 신규 회원에게 불법으로 리크루팅 쪽지/메일을 보내는 분들이
있습니다. 불법광고로 입사한 회원들이 100% 피해를 보고 있습니다.
내용 캡쳐해서 제게 문자 주시면 조치하고 커피쿠폰 드리겠습니다.
운영자 조이파파 010-5582-7573※※※
--- 게시글 수집 완료 ---

사용자: 스고이
내용:
※제목앞에 말머리를 꼭 달아주세요 (질문 / 고민 / 일상/ 영업일기)
손해보험 설계사시험 요약정리된것 있으신분 계실까요..?
시험보실분이 계신데 시간이 별로 없어서 요약집 찾고있습니다ㅜㅜ
--- 게시글 수집 완료 ---

사용자: 딸램셋파파
내용:
1. 먼저 이 글의 제목을 변경 해 주세요 : 지역+

사용자: 머니쏭
내용:
※제목앞에 말머리를 꼭 달아주세요 (질문 / 고민 / 일상/ 영업일기)
위 문제의 답이 2번이라는데 이유를 모르겠어요
고수님들 도와주세뇨 내일이시험이에요..
--- 게시글 수집 완료 ---

사용자: 퐁퐁펑
내용:
1. 먼저 이 글의 제목을 변경 해 주세요 : 지역+회사명+보험설계사+닉네임
(예시) 수원 ㅇㅇ보험회사/ ㅇㅇGA 보험설계사 뽀빠이 가입인사 합니다.
2. 카페 활동 닉네임 (영어이신 분은 한글로 바꾸어주세요) : 퐁퐁펑
3. 소속 보험회사 : GA
4. 보험경력 : 신입
5. 보만세에서 얻고 싶은 것 : 다같이 정보공유도하고 교육자료도 얻고싶어요~
6. 자기소개(경력, 영업방법 등) : 이제 막 시험합격했어요ㅎㅎ
7. 가입경로 [ 예) 검색(검색어 입력-보험설계사 등등), 소개, 기타 ] : 보험설계사
●필독 - 회원등업
https://cafe.naver.com/gustn34264/16449
가입인사글1개+댓글20개 정회원 자동등업
★등업혜택 : 필수자료 150개★
https://cafe.naver.com/gustn34264/56557
● 카톡 채널을 "추가"하시면
유용한 정보와 안내를 받아 보실 수 있습니다.
(카톡에서 '보만세'검색 또는
채널링크 접속 : http://pf.kakao.com/_rtxjyb )
● 위 질문 미 입력시 다시 신입회원으로 강등.
★이 달의 보험강의 모음★
https://cafe.naver.com/gustn34264/53907
📢보만세 광고안내
https://cafe.naver.com/gustn34264/16547
※※※ 신규 회원에게 불법으로 리크루팅 쪽지/메일을 보내는 분들이
있습니다. 불법광고로 입사한 회원들이 100% 피해를 보고 있습니다.
내용 캡쳐해서 제게 문자 주시면 조치하고 커피쿠폰 드리겠습니다.
운영자 조이파파 010-5582-7573※※※
--- 게시글 수집 완료 ---

사용자: 경이01
내용:
1. 먼저 이 글의 제목을 변경 해 주세요 : 지역+회사명+보험설계사+닉네임
(예시

# 보만사 2년치

In [83]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time
import pandas as pd
import json
from dateutil.relativedelta import relativedelta # pip install python-dateutil 필요

# --- WebDriver 설정 ---
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("lang=ko_KR")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# --- 로그인 과정 ---
driver.get("https://cafe.naver.com/f-e/cafes/19997929/menus/0?q=%EC%8B%9C%ED%97%98&t=1750149314419")
wait = WebDriverWait(driver, 20) 

try:
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gnb_login_button > span.gnb_txt')))
    login_button.click()
    print("로그인 버튼 클릭 성공.")
    time.sleep(1)

    id_input_element = wait.until(EC.presence_of_element_located((By.ID, 'id')))
    driver.execute_script("arguments[0].focus();", id_input_element)
    time.sleep(0.5)
    driver.execute_script("arguments[0].value = '아이디';", id_input_element)
    print("ID 입력 성공.")

    pw_input_element = wait.until(EC.presence_of_element_located((By.ID, 'pw')))
    driver.execute_script("arguments[0].focus();", pw_input_element)
    time.sleep(0.5)
    driver.execute_script("arguments[0].value = '비밀번호';", pw_input_element)
    print("PW 입력 성공.")
    time.sleep(1)

    final_login_button = driver.find_element(By.CSS_SELECTOR, "#log\\.login")
    final_login_button.click()
    print("최종 로그인 버튼 클릭 성공.")
    time.sleep(5) 

except Exception as e:
    print(f"로그인 과정 중 오류 발생: {e}")
    driver.quit()
    exit()

# --- 데이터 수집 준비 ---
all_posts_data = []

# 2년 전 날짜 계산 (python-dateutil의 relativedelta 사용)
two_years_ago = datetime.now() - relativedelta(days=2) 
print(f"데이터 수집 기준 날짜 (2년 전): {two_years_ago.strftime('%Y-%m-%d %H:%M:%S')}")

break_outer_loop = False 

# --- 게시글 및 댓글 수집 루프 ---
while True:
    if break_outer_loop:
        break

    print(f"\n--- 새 페이지 로딩 중 ({driver.current_url}) ---")
    
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody > tr:nth-child(1) > td:nth-child(2) > div > div > a.article")))
        print("게시글 목록 로드 완료.")
    except Exception as e:
        print(f"게시글 목록 로딩 실패 (페이지에 게시글이 없거나 구조 변경): {e}")
        break_outer_loop = True 
        break

    for i in range(1, 16): 
        if break_outer_loop: 
            break

        current_post_user = ""
        current_post_content = ""
        parsed_post_date = None
        current_comments = [] # 각 게시글의 댓글을 저장할 리스트 초기화

        try:
            # 게시글 버튼 찾기 및 클릭
            article_selector = f"tbody > tr:nth-child({i}) > td:nth-child(2) > div > div > a.article"
            article_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, article_selector)))
            
            driver.execute_script("arguments[0].scrollIntoView(true);", article_button)
            article_button.click()
            print(f"게시글 {i} 클릭 성공.")
            time.sleep(3) 

            # 게시글 내용과 댓글이 있는 iframe으로 전환 (cafe_main)
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "cafe_main")))
            print("iframe 전환 성공.")

            # --- 게시글 기본 정보 추출 (iframe 내부) ---
            content_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#app > div > div > div.ArticleContentBox > div.article_container > div:nth-child(1)")))
            current_post_content = content_element.text.strip()
            if "●" in current_post_content:
                current_post_content = current_post_content.split("●")[0].strip()
            
            user_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div.profile_area > div.profile_info > div")))
            current_post_user = user_element.text.strip()

            post_date_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#app > div > div > div.ArticleContentBox > div.article_header > div.ArticleInfo > div.info_data > span.date")))
            post_date_str = post_date_element.text.strip()
            
            # --- 게시글 작성일 파싱 ---
            current_year = datetime.now().year
            try:
                if len(post_date_str) == 5 and '.' not in post_date_str: 
                    parsed_post_date = datetime.now().replace(hour=int(post_date_str[:2]), minute=int(post_date_str[3:]), second=0, microsecond=0)
                elif '어제' in post_date_str:
                    parsed_post_date = datetime.now() - timedelta(days=1)
                    parsed_post_date = parsed_post_date.replace(hour=int(post_date_str[-5:-3]), minute=int(post_date_str[-2:]), second=0, microsecond=0)
                elif len(post_date_str) == 11 and '.' in post_date_str and post_date_str.count('.') == 2: 
                    parsed_post_date = datetime.strptime(f"{current_year}.{post_date_str}", "%Y.%m.%d. %H:%M")
                else:
                    parsed_post_date = datetime.strptime(post_date_str, "%Y.%m.%d. %H:%M")
            except ValueError as ve_date:
                print(f"게시글 날짜 파싱 오류: '{post_date_str}' - {ve_date}. 이 게시글은 날짜 없이 처리됩니다.")
                parsed_post_date = None

            # --- 2년치 데이터 필터링 (날짜가 오래되면 수집 중단) ---
            if parsed_post_date and parsed_post_date < two_years_ago:
                print(f"게시글 작성일({parsed_post_date.strftime('%Y-%m-%d %H:%M:%S')})이 2년 전({two_years_ago.strftime('%Y-%m-%d %H:%M:%S')})보다 오래되어 수집을 중단합니다.")
                break_outer_loop = True 
                break # 내부 루프 종료 (다음 게시글로 가지 않음)
            
            print(f"게시글 정보 - 사용자: {current_post_user}, 내용: {current_post_content[:50]}..., 작성일: {parsed_post_date}")

            # --- 댓글 수집 로직 (iframe 내부에서 직접 수집) ---
            try:
                # '댓글 더보기' 버튼이 있다면 클릭하여 모든 댓글 로드 (iframe 내부에서 찾음)
                try:
                    more_comments_button = driver.find_element(By.CSS_SELECTOR, "a.btn_rc_more") 
                    if more_comments_button.is_displayed():
                        more_comments_button.click()
                        print("댓글 '더보기' 버튼 클릭.")
                        time.sleep(2) 
                except:
                    pass # '더보기' 버튼이 없으면 무시

                comment_items = driver.find_elements(By.CSS_SELECTOR, "div.CommentBox > div:nth-child(2) > ul > li") 
                
                if comment_items:
                    print(f"총 {len(comment_items)}개의 댓글 발견.")
                    for item in comment_items:
                        try:

                            comment_text_span = item.find_element(By.CSS_SELECTOR, "div.comment_text_box > p > span").text
                            comment_text = comment_text_span.strip()
                            
                        except Exception as e_comment:
                            print(f"개별 댓글 추출 중 오류 발생: {e_comment}")
                            continue 
                else:
                    print("댓글 없음.")
            except Exception as e_comments_load:
                print(f"댓글 로딩 또는 처리 중 오류 발생: {e_comments_load}")
            # --- 댓글 수집 로직 끝 ---
            
            # 추출된 데이터를 리스트에 추가
            all_posts_data.append({
                "User": current_post_user,
                "Content": current_post_content,
                "PostDate": parsed_post_date.strftime("%Y-%m-%d %H:%M:%S") if parsed_post_date else "N/A",
                "Comments": current_comments 
            })
            print("--- 게시글 수집 완료 ---\n")

        except Exception as e:
            print(f"게시글 {i} 수집 중 오류 발생: {e}")
            # 게시글 수집 중 오류가 발생했어도 현재 페이지의 나머지 게시글은 시도해야 할 수 있으므로
            # 여기서는 break만 하고, 페이지네이션 루프는 계속 진행합니다.
            # 만약 다음 게시글도 문제가 있다면, 다시 이 except 블록에 들어올 것입니다.
            # 다만, iframe에 갇히는 것을 방지하기 위해 default_content로 돌아가는 것은 필요합니다.
            try:
                driver.switch_to.default_content()
                time.sleep(1)
            except Exception as e_iframe_switch:
                print(f"오류 후 iframe에서 벗어나기 실패: {e_iframe_switch}")
                # 이 경우 복구가 어려우므로 전체 루프를 종료합니다.
                break_outer_loop = True
                break
            break # 현재 페이지의 게시글 순회 중단하고 다음 페이지네이션으로 넘어감
            
        finally:
            # 게시글 상세 페이지 작업이 끝났으므로 (오류 여부와 무관하게)
            # iframe에서 벗어나고 게시글 목록 페이지로 돌아갑니다.
            if not break_outer_loop: # 이미 전체 루프 종료 플래그가 설정된 경우는 제외
                try:
                    # 현재 프레임이 'cafe_main'인지 확인하고 전환
                    current_frame_id = driver.execute_script("return window.frameElement ? window.frameElement.id : null;")
                    if current_frame_id == "cafe_main":
                        driver.switch_to.default_content()
                        time.sleep(1)
                    
                    driver.back() # 게시글 목록 페이지로 돌아가기
                    time.sleep(2) 
                except Exception as e_finally:
                    print(f"finally 블록에서 이전 페이지로 이동 실패: {e_finally}")
                    break_outer_loop = True # 복구 실패 시 전체 루프 종료
                    break


    # --- 페이지네이션 처리 ---
    if break_outer_loop: 
        break

    try:
        page_numbers_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".Pagination .btn.number")))
        
        next_page_button_to_click = None
        current_page_num = -1

        for btn in page_numbers_elements:
            if btn.get_attribute("aria-pressed") == "true":
                current_page_num = int(btn.text)
                break
        
        found_next_in_set = False
        for btn in page_numbers_elements:
            try:
                btn_num = int(btn.text)
                if btn_num == current_page_num + 1:
                    next_page_button_to_click = btn
                    found_next_in_set = True
                    break
            except ValueError:
                continue
        
        if not found_next_in_set:
            try:
                next_set_button = driver.find_element(By.CSS_SELECTOR, ".Pagination .btn.type_next")
                
                if "disabled" in next_set_button.get_attribute("class"):
                    print("마지막 페이지 세트입니다. 더 이상 페이지가 없습니다.")
                    break_outer_loop = True 
                    break 
                
                next_page_button_to_click = next_set_button
                print("다음 페이지 세트로 이동합니다.")
                
            except Exception:
                print("더 이상 페이지를 찾을 수 없습니다. (다음 세트 버튼 없음)")
                break_outer_loop = True 
                break

        if next_page_button_to_click:
            wait.until(EC.element_to_be_clickable(next_page_button_to_click)).click()
            print(f"페이지 이동: {next_page_button_to_click.text if next_page_button_to_click.text else '다음 세트'}")
            time.sleep(4) 
        else:
            print("더 이상 이동할 페이지가 없어 종료합니다.")
            break_outer_loop = True 
            break

    except Exception as e_pagination:
        print(f"페이지네이션 처리 중 오류 발생 또는 마지막 페이지 도달: {e_pagination}")
        break_outer_loop = True 
        break

# WebDriver 종료
driver.quit()
print("WebDriver 종료.")


## CSV 파일로 데이터 추출

now = datetime.now()
file_name = f"naver_cafe_posts_{now.strftime('%Y%m%d_%H%M%S')}.csv"

df = pd.DataFrame(all_posts_data)

# 'Comments' 컬럼의 리스트 오브 딕셔너리를 JSON 문자열로 변환하여 저장
df['Comments'] = df['Comments'].apply(lambda x: json.dumps(x, ensure_ascii=False))

df.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"\n총 {len(all_posts_data)}개의 게시글 데이터를 '{file_name}'으로 저장했습니다.")
print(f"CSV 파일 '{file_name}'이 생성되었습니다.")

로그인 버튼 클릭 성공.
ID 입력 성공.
PW 입력 성공.
최종 로그인 버튼 클릭 성공.
데이터 수집 기준 날짜 (2년 전): 2025-06-16 13:41:25

--- 새 페이지 로딩 중 (https://cafe.naver.com/f-e/cafes/19997929/menus/0?q=%EC%8B%9C%ED%97%98&t=1750149314419) ---
게시글 목록 로드 완료.
게시글 1 클릭 성공.
iframe 전환 성공.
게시글 1 수집 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0x1213b03+62899]
	GetHandleVerifier [0x0x1213b44+62964]
	(No symbol) [0x0x10410f3]
	(No symbol) [0x0x108980e]
	(No symbol) [0x0x1089bab]
	(No symbol) [0x0x10d25c2]
	(No symbol) [0x0x10ae554]
	(No symbol) [0x0x10cfd81]
	(No symbol) [0x0x10ae306]
	(No symbol) [0x0x107d670]
	(No symbol) [0x0x107e4e4]
	GetHandleVerifier [0x0x1474793+2556483]
	GetHandleVerifier [0x0x146fd02+2537394]
	GetHandleVerifier [0x0x123a2fa+220586]
	GetHandleVerifier [0x0x122aae8+157080]
	GetHandleVerifier [0x0x123141d+184013]
	GetHandleVerifier [0x0x121ba68+95512]
	GetHandleVerifier [0x0x121bc10+95936]
	GetHandleVerifier [0x0x1206b5a+9738]
	BaseThreadInitThunk [0x0x764e5d49+25]
	RtlInitializeExceptionChain [0x0x77dbd0

KeyError: 'Comments'